In [56]:
# !pip install xgboost
# !pip install shap
# !conda install -y -c dglteam dgl-cuda10.1
# !conda install -y pytorch torchvision cudatoolkit=10.1 -c pytorch

In [50]:
import pickle
import xgboost as xgb
import shap

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.datasets import load_iris, load_digits, load_boston

In [52]:
df = pd.read_csv('/data/results/ucec_inputs.tsv', sep='\t', index_col='Unnamed: 0')
df

,AAGAB:NP_078942.3:K290_acetylome,ABCE1:NP_001035809.1:K431_acetylome,ACAA2:NP_006102.2:K13_acetylome,ACAA2:NP_006102.2:K137_acetylome,ACAA2:NP_006102.2:K234_acetylome,ACADM:NP_001272972.1:K312_acetylome,ACADVL:NP_001257376.1:K262_acetylome,ACADVL:NP_001257376.1:K299_acetylome,ACADVL:NP_001257376.1:K301_acetylome,ACAT1:NP_000010.1:K124_acetylome,...,ZSWIM8_proteome,ZW10_proteome,ZWILCH_proteome,ZWINT_proteome,ZXDC_proteome,ZYG11B_proteome,ZYX_proteome,ZZEF1_proteome,ZZZ3_proteome,sample_type
S001,0.46100,-0.22700,-1.0700,-1.030,-1.0400,2.41000,1.070,1.930,1.670,0.5430,...,-0.08770,0.0229,0.1090,-0.000315,-0.332,-0.43300,-1.020,-0.1230,-0.08590,tumor
S002,1.77000,0.56400,-1.0300,-0.906,-0.6040,-0.55700,-0.329,0.360,-0.586,0.1670,...,-0.03560,0.3630,1.0700,0.737000,-0.564,-0.00461,-1.130,-0.0757,-0.47300,tumor
S003,-0.81500,0.10500,0.0573,0.719,0.6040,0.65100,1.160,1.100,0.902,-0.6980,...,0.00112,0.0105,-0.1160,-0.000315,0.151,-0.07400,-0.540,0.3200,-0.41900,tumor
S005,-0.00334,-0.04860,-0.7900,-0.949,-0.3210,0.70400,0.934,1.250,1.190,-0.1600,...,0.07250,-0.0714,0.0933,0.156000,-0.398,-0.07520,-0.797,-0.0301,-0.46700,tumor
S006,0.20500,0.48900,0.6510,0.434,0.0320,-0.45000,0.111,0.247,-0.441,-0.0157,...,-0.17600,-1.2200,-0.5620,0.937000,-0.646,0.20700,-1.850,-0.1760,0.05130,tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S149,-0.75700,-0.04345,0.0604,0.206,-0.5580,0.00757,-0.505,-0.715,-0.804,-0.0119,...,-0.20400,-0.1510,0.0552,0.105000,1.330,-0.67400,-0.840,-0.2690,0.10300,normal
S150,-0.72800,-0.04345,-0.0492,-0.138,-0.2200,0.00757,-0.116,0.117,-0.313,0.0169,...,-0.07820,0.1410,0.7260,0.868000,0.662,-0.57000,-0.399,-0.0959,-0.06350,normal
S151,-1.16000,-0.04345,0.2690,0.416,-0.2380,0.00757,-0.140,-0.170,-0.414,0.0519,...,0.12200,-0.3800,-0.2970,0.046200,0.045,-0.31900,0.835,0.1690,-0.00809,normal
S152,-0.59600,-0.04345,0.0210,0.282,-0.0356,0.00757,-0.517,-0.815,-0.752,-0.0125,...,0.06070,-0.1260,-0.0662,-0.010300,1.050,-0.35600,0.416,0.2730,-0.65800,normal


In [35]:
X,y = shap.datasets.boston()
# model = xgb.train({"learning_rate": 0.01}, xgb.DMatrix(X, label=y), 100)

In [36]:
train_idxs = np.random.choice(np.arange(X.shape[0]), replace=False, size=int(X.shape[0] * .8)).flatten()
val_idxs = np.asarray([x for x in np.arange(X.shape[0]) if x not in train_idxs])

X_train, y_train = X.iloc[train_idxs, :], y[train_idxs]
X_val, y_val = X.iloc[val_idxs, :], y[val_idxs]

In [37]:
X

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [43]:
clf = xgb.XGBRegressor(random_state=42)
model = clf.fit(X_train, y_train)

In [44]:
# model = xgb.fit({"learning_rate": 0.01}, xgb.DMatrix(X[train_idxs], label=y[train_idxs]), 100)

In [45]:
booster = model.get_booster()

model_bytearray = booster.save_raw()[4:]
def myfun(self=None):
    return model_bytearray
booster.save_raw = myfun

# Shap explainer initilization
# shap_ex = shap.TreeExplainer(mybooster)

In [46]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

In [48]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0,:], X_train.iloc[0,:])

In [49]:
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values, X)